In [ ]:
!git clone https://github.com/SpaceTimeNarratives/demo.git

Cloning into 'demo'...
remote: Enumerating objects: 315, done.
remote: Counting objects: 100% (315/315), done.
remote: Compressing objects: 100% (242/242), done.
remote: Total 315 (delta 138), reused 205 (delta 64), pack-reused 0
Receiving objects: 100% (315/315), 29.19 MiB | 12.95 MiB/s, done.
Resolving deltas: 100% (138/138), done.


In [ ]:
cd demo

/content/demo


Import libraries

In [ ]:
import re
import string
import nltk
from IPython.display import HTML
nltk.download('punkt')

Define functions

In [238]:
# Storing the sets of punctuation in variable result


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [277]:
BG_COLOR = {
    'PLNAME':'#feca74','GEONOUN': '#9cc9cc', 'GPE':'#feca74', 'CARDINAL':'#e4e7d2',
    'FAC':'#9cc9cc','QUANTITY':'#e4e7d2','PERSON':'#aa9cfc', 'ORDINAL':'#e4e7d2',
    'ORG':'#7aecec', 'NORP':'#d9fe74', 'LOC':'#9ac9f5', 'DATE':'#c7f5a9',
    'PRODUCT':'#edf5a9', 'EVENT': '#e1a9f5','TIME':'#a9f5bc', 'WORK_OF_ART':'#e6c1d7',
    'LAW':'#e6e6c1','LANGUAGE':'#c9bdc7', 'PERCENT':'#c9ebf5', 'MONEY':'#b3d6f2',
    'EMOTION':'#f2ecd0', 'TIME-SEM':'#d0e0f2', 'MOVEMENT':'#f2d0d0','no_tag':'#FFFFFF'
}

# Sorts the dictionary keys alphabetically
def sort_dict(a_dict):
  return {i:a_dict[i] for i in sorted(a_dict.keys())}

# Converts a string to a regular expression pattern that matches the string in uppercase or lowercase
def to_regex(string):
    pattern = ""
    for i, char in enumerate(string):
        pattern += f"[{char.upper()}{char.lower()}]" if i>0 else f"[{char.upper()}]"
    return pattern

# Generates a list of all tokens, tagged and untagged, for visualization
def get_tagged_list(text, entities):
  begin, tokens_tags = 0, []
  for start, (ent, tag) in entities.items():
    if begin <= start:
      tokens_tags.append((text[begin:start], None))
      tokens_tags.append((text[start:start+len(ent)], tag))
      begin = start+len(ent)
  tokens_tags.append((text[begin:], None)) #add the last untagged chunk
  return tokens_tags

# Marking up the token for visualization
def mark_up(token, tag=None):
  if tag:
    begin_bkgr = f'<bgr class="entity" style="background: {BG_COLOR[tag]}; padding: 0.05em 0.05em; margin: 0 0.15em;  border-radius: 0.4em;">'
    end_bkgr = '\n</bgr>'
    begin_span = '<span style="font-size: 0.8em; font-weight: bold; border-radius: 0.35em; vertical-align: middle; margin-left: 0.5rem">'
    end_span = '\n</span>'
    return f"{begin_bkgr}{token}{begin_span}{tag}{end_span}{end_bkgr}"
  return f"{token}"

# generate html formatted text 
def visualize(text, entities):
  token_tag_list = get_tagged_list(text, entities)
  start_div = f'<div class="entities" style="line-height: 2.0; direction: ltr"><p>'
  end_div = '</p></div>'
  html = start_div
  # html = ''
  for token, tag in token_tag_list:
    html += mark_up(token,tag)
  html += end_div
  return html

# Define regular expression pattern to match place names
# Loop through each sentence and find matches
def visualize_sents(sent_list, sent_start_index=None, sent_end_index=None, index_list=None, sent_with_entities_only=False):
  if index_list is None:
    index_list = range(len(sent_list))
    if sent_start_index is not None:
      if sent_end_index is not None:
        index_list = range(sent_start_index, sent_end_index+1)
      else:
        index_list = range(sent_start_index, len(sent_list))
    elif sent_end_index is not None:
      index_list = range(sent_end_index+1)

  html = ''
  for i in index_list[:50]:
    sentence = sent_list[i]
    sentence_id = f"<b>Sentence {i}:</b> {sentence}"
    found_spans, entities = [], {}
    for name in place_names:
      for match in re.finditer(f'{name}|{name.upper()}', sentence_id):
        start_index = match.start()
        if start_index not in found_spans:
            found_spans += list(range(start_index, start_index+len(name)+1))
            entities[start_index] = name, 'PLNAME'
    if entities or not sent_with_entities_only:
        html += visualize(sentence_id, sort_dict(entities))
  return HTML(html)

Get example text

In [268]:
text = """From Penrith two roads lead to Pooley Bridge, about six miles distant, which spans the Eamont just at its issue from Ulleswater. Either road may be taken, be we recommend that which follows the Shap road to Eamont Bridge. Carleton Hall is near to it on the left. Cross the bridge, and take the first road to the right. At this point, on the left, are the druidical remains called King Arthur's Round Table, and Mayborough. Immediately after crossing Pooley Bridge, the road runs along the western shore of ULLESWATER.
To Patterdale, a distance of ten miles; but, before proceeding along it, the tourist would do well to take a walk of a few miles along the eastern shore, in the direction of Martindale, from several points on which he will obtain a good view of the lake. Should this deviation be made, it will be necessary to return by the same road to Pooley Bridge, where there are two small inns, at which boats, for an excursion on the water, or for fishing, may be procured if desired. A fine view may be had from the top of Dunmallet, on which are the vestiges of a Roman fort. 
There is some good fishing here in the lake and stream. Trout, perch, and eels, are numerous, and large lake-trout are sometimes, though seldom, taken. Lowther Castle, the seat of the Earl of Lonsdale, is distant from this bridge about four miles. Ulleswater is nine miles in length, by nearly a mile wide, at the broadest point; but, owing to its irregular form, it is divided into three reaches, the first of which is closed in by Hallen Fell, on the western shore."""

In [275]:
# Split the text into paragraphs
# paragraphs = text.split("\n")
sentences = nltk.sent_tokenize(text)

# List of known place names to check against
place_names = [name.strip().title().replace("'S", "'s") for name in open('LD_placenames.txt').readlines()]
place_names = sorted(set(place_names), key=lambda x:len(x), reverse=True)

len(place_names)

2412

Test function calls (uncomment)

In [278]:
# visualize_sents(sentences)
# visualize_sents(sentences, sent_start_index=3)
# visualize_sents(sentences, sent_end_index=7)
# visualize_sents(sentences, sent_start_index=3, sent_end_index=7)
# visualize_sents(sentences, index_list=[2, 5, 7, 10])
# visualize_sents(sentences, sent_start_index=3, sent_end_index=7, index_list=[2, 5, 7, 10], sent_with_entities_only)
visualize_sents(sentences, sent_with_entities_only=True)

Moving on to bigger files...

In [ ]:
def clean_text(input_text):
  # Define a regular expression pattern to match XML tags
  pattern = re.compile(r'<.*?>')

  # Use the sub() function to remove all tags from the XML text
  clean_text = re.sub(pattern, '', input_text)

  # use the nltk sentence tokenizer to segment the text into sentences
  clean_text = clean_text.replace('\n', ' ').replace('\t', ' ')

  # Replace multiple spaces with one space
  clean_text = re.sub(r'\s+', ' ', clean_text)

  # Split into sentences, strip leading and trailing non-printables and return 
  return [s.strip() for s in nltk.sent_tokenize(clean_text)]

In [280]:
# Open the XML file, clean and split content into sentences
with open('gold_standard/Anon_cqp_66.xml', 'r') as file:
    sentences = clean_text(file.read())
# visualize_sents(sentences)
# visualize_sents(sentences, sent_start_index=3)
# visualize_sents(sentences, sent_end_index=7)
# visualize_sents(sentences, sent_start_index=3, sent_end_index=7)
# visualize_sents(sentences, index_list=[2, 5, 7, 10])
# visualize_sents(sentences, sent_start_index=3, sent_end_index=7, index_list=[2, 5, 7, 10], sent_with_entities_only)
visualize_sents(sentences, 100, 150, sent_with_entities_only=True)

Reading the `LDv5_corpus_pl_names.xlsx`

```
clusters_para_all.py
```



In [ ]:
### clusters_para_all.py ###
# Import the CSV module for working with CSV files
import csv

# Define a class to store coordinates (latitude and longitude)
# If a coordinate value cannot be converted to float, it will be stored as a string
class TopoCoord:
    def __init__(self, lat, long):
        try:
            self.lat = float(lat)
            self.long = float(long)
        except ValueError:
            self.lat = lat
            self.long = long
            print(f"Error in coordinates: {lat}, {long}")

# This function reads in a file containing place names and their coordinates
# It returns a dictionary where each key is a unique place name
# and each value is a TopoCoord object containing the place's latitude and longitude
def setNameCoords(placeNamesFileName, stNameColNo=8, latColNo=5, longColNo=4):
    # Create an empty dictionary to store place name coordinates
    nameCoords = {}
    # Open the input file
    with open(placeNamesFileName, encoding="utf-8") as f:
        # Use the CSV module to read in each row
        csvReader = csv.reader(f, delimiter=",")
        # Skip the first row (header row)
        firstRow = True
        # For each subsequent row, extract the place name and coordinates
        for row in csvReader:
            if firstRow:
                firstRow = False
            else:
                stName = row[stNameColNo]
                # If this is the first time we've seen this place name, add it to the dictionary
                if stName not in nameCoords.keys():
                    nameCoords[stName] = TopoCoord(row[latColNo], row[longColNo])
    # Print the number of unique place names that were read in
    print(f"{len(nameCoords.keys())} unique names read in from {placeNamesFileName}")
    # Return the dictionary of place name coordinates
    return nameCoords

# This function checks whether a given latitude and longitude is within the bounding box of the Lake District
# It returns True if the coordinates are within the box, and False otherwise
def isInLD(lat, long):
    # Define the bounding box coordinates for the Lake District
    # This version uses the coordinates from the paper "Automatic Extraction of Walk Descriptions from Blog Posts" by AAAG (http://aclweb.org/anthology/E17-3010)
    LDBBox = [-4.60, 54.0, -2.50, 55.0]
    # Check if the given coordinates are within the bounding box
    try:
        if float(long) >= float(LDBBox[0]) and float(long) <= float(LDBBox[2]):
            if float(lat) >= float(LDBBox[1]) and float(lat) <= float(LDBBox[3]):
                return True
    except ValueError:
        print(f"SN_tools isInLD could not locate {lat}, {long}")
    return False

# This function reads in a file and returns a dictionary of the first sentence number for each paragraph
# The keys are paragraph numbers (starting at 1)
# The values are the sentence number of the first sentence in each paragraph
def setFirstSent(inFileName, firstSent={}):
    # Define the HTML tag that indicates the start of a paragraph
    paraStart = '<p><s id="s'
    # An alternative start of a para (used in some files)
    altStart = '<p> <s id="s'
    # Initialize the paragraph number to 1

```
kmeans_annotated.py
```

In [ ]:
def initk(points, k, init):
    xmin, ymin = min(p.x for p in points), min(p.y for p in points)
    xmax, ymax = max(p.x for p in points), max(p.y for p in points)
    nearests=[[] for i in range(k)]
    while [] in nearests:
        if init=="forgy":
            means=random.sample(points, k)
        elif init=="random":
            means=[Point(random.uniform(xmin, xmax), random.uniform(ymin,ymax)) for i in range(k)]
        else:
            print("Error: unknown initialisation method")
            sys.exit(1)
        nearests, totaldist=clustering_dist(points, means)
    return means, nearests, totaldist


def kmeans(points, k, threshold=5, init="forgy"):
    bigdiff=True
    means, nearests, totaldist=initk(points, k, init)
    cnt=1
    while bigdiff:
        means2 = [Point(sum(p.x for p in points[ii] in nearests[j])/len(nearests[j]), 
                        sum(p.y for p in points[ii] in nearests[j])/len(nearests[j]))
                  for j in range(k)]
        nearests, newtotal=clustering_dist(points, means2)
        offset=totaldist-newtotal
        cnt+=1
        if offset>threshold:
            means=means2
            totaldist=newtotal
        else:
            bigdiff=False
    return totaldist, means, nearests


In [ ]:
# Extract entities from text using spaCy
doc = nlp(text)
entities = [(entity.text, entity.label_) for entity in doc.ents]

# Filter out entities that are not locations
locations = []
for entity in entities:
    if entity[1] in ['GPE', 'PERSON', 'ORG']:
        locations.append(entity[0])

# Get coordinates for locations using GeoPy
geolocator = Nominatim(user_agent="my-application")
coordinates = []
for location in locations:
    location_data = geolocator.geocode(location)
    if location_data is not None:
        coordinates.append((location_data.latitude, location_data.longitude))

# Create map using Folium and add markers for locations
m = folium.Map(location=[coordinates[0][0], coordinates[0][1]], zoom_start=5)
for coord in coordinates:
    folium.Marker(location=[coord[0], coord[1]]).add_to(m)

# Display map
m


In [ ]:
import spacy
from spacy import displacy

# Load the NER model
nlp = spacy.load("en_core_web_sm")

# Define a function to extract entities and their relationships from text
def extract_entities(text):
    doc = nlp(text)
    entities = []
    relations = []

    # Extract named entities
    for ent in doc.ents:
        if ent.label_ == "GPE" or ent.label_ == "LOC":
            entities.append(ent)

    # Extract relationships between named entities
    for chunk in doc.noun_chunks:
        subj = None
        obj = None
        for ent in entities:
            if ent.text == chunk.text:
                if not subj:
                    subj = ent
                else:
                    obj = ent
                    break
        if subj and obj:
            relations.append((subj, obj))

    return entities, relations


# Define a function to disambiguate locations
def disambiguate_locations(entities):
    # Create a set of unique location names
    unique_locs = set()
    for entity in entities:
        unique_locs.add(entity.text)

    # Disambiguate location names
    for loc in unique_locs:
        count = 0
        for entity in entities:
            if entity.text == loc:
                count += 1
                if count > 1:
                    entity._.is_ambiguous = True
                else:
                    entity._.is_ambiguous = False

# Define a function to plot entities and their relationships on a map
def plot_map(entities, relations):
    options = {"ents": ["GPE", "LOC"], "colors": {"GPE": "orange", "LOC": "green"}}
    if entities:
        disambiguate_locations(entities)
        displacy.render(entities, style="ent", jupyter=True, options=options)
    if relations:
        displacy.render(relations, style="dep", jupyter=True, options=options)

# Extract entities and their relationships from multiple sentences
sentences = ["I live in New York and work in Boston.", "My friend works in Chicago but also visits Los Angeles."]
entities, relations = [],[]
for sentence in sentences:
    sent_entities, sent_relations = extract_entities(sentence)
    entities += sent_entities
    relations += sent_relations

# Plot entities and their relationships on a map
plot_map(entities, relations)

AttributeError: ignored

In [ ]:
import geopy
import folium

# Define the bounding box
bbox = [(54.0, -4.60), (55.0, -2.50)]

# Define the list of place names
places = ['London', 'Edinburgh', 'Birmingham', 'Bristol', 'Cardiff', 'Glasgow']

# Create a map centered on the bounding box with smaller width and height
map = folium.Map(location=[(bbox[0][0]+bbox[1][0])/2, (bbox[0][1]+bbox[1][1])/2], zoom_start=8, tiles='Stamen Toner', width=500, height=300)

# Create two feature groups to store the markers for the places within and outside the bounding box
inside_bbox = folium.FeatureGroup(name='Inside Bounding Box')
outside_bbox = folium.FeatureGroup(name='Outside Bounding Box')

# Loop through each place name and get its coordinates
for place in places:
    location = geopy.geocoders.Nominatim(user_agent="my-application").geocode(place)
    lat, lon = location.latitude, location.longitude
    
    # Check if the coordinates are within the bounding box
    if bbox[0][1] <= lon <= bbox[1][1] and bbox[0][0] <= lat <= bbox[1][0]:
        # Add a marker to the inside_bbox feature group
        folium.Marker(location=[lat, lon], tooltip=place, icon=folium.Icon(color='green', icon='info-sign'), popup=place).add_to(inside_bbox)
    else:
        # Add a marker to the outside_bbox feature group
        folium.Marker(location=[lat, lon], tooltip=place, icon=folium.Icon(color='red', icon='info-sign'), popup=place).add_to(outside_bbox)

# Add the feature groups to the map
inside_bbox.add_to(map)
outside_bbox.add_to(map)

# Add a layer control to toggle between the feature groups
folium.LayerControl().add_to(map)

# Display the map
map

# Part 2